<a href="https://colab.research.google.com/github/gul-dan-van/CSS-popup/blob/main/minor2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [6]:
! mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip train.zip

In [ ]:
!unzip test1.zip

In [14]:
import os
import numpy as np
from random import shuffle
import cv2
from tqdm import tqdm


TRAIN_DIR="train"
TEST_DIR="test1"
IMG_SIZE=50
LR=1e-3

MODEL_NAME=f"dogsvscats-{LR}-{'2conv-basic'}.model"

In [15]:
def label_img(img):
  label=img.split('.')[0]
  if label=='dog':return [0,1]
  else:return [1,0]

In [16]:
def create_train_data():
  training_data=[]
  for img in os.listdir(TRAIN_DIR):
    label=label_img(img)
    path=TRAIN_DIR+'/'+img
    img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
    training_data.append( [np.array(img), np.array(label)] )
  shuffle(training_data)
  np.save('train_data.npy',training_data)
  return training_data

In [17]:
def create_test_data():
  testing_data=[]
  for img in os.listdir(TEST_DIR):
    label=label_img(img)
    path=TEST_DIR+'/'+img
    IMG_NUM=img.split('.')[0]
    img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
    testing_data.append( [np.array(img), IMG_NUM] )
  shuffle(testing_data)
  np.save('test_data.npy',testing_data)
  return testing_data

In [ ]:
if 'train_data.npy' in os.listdir():
  train_data=np.load('train_data.npy')
else:
  train_data=create_train_data()

In [ ]:
!pip install tflearn

In [45]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir="log")

In [38]:
if os.path.exists(f"{MODEL_NAME}.meta"):
  model.load(MODEL_NAME)

In [22]:
train=train_data[:1000]
test=train_data[1000:]

In [23]:
X=np.array([x[0]for x in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y=[x[1]for x in train]

test_x=np.array([x[0]for x in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y=[x[1]for x in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [ ]:
if 'test_data.npy' in os.listdir():
  final_test=np.load('test_data.npy')
else:
  final_test=create_test_data()

In [36]:
img_data=cv2.resize(cv2.imread("test1/42.jpg",cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
# img_data=test_data[0]
data=img_data.reshape(IMG_SIZE,IMG_SIZE,1)
print(model.predict([data])[0][0])

0.5951243


In [ ]:
with open('test_results.csv','w') as f:
  f.write('id,label\n')
  s=0
  n=0
  for data in final_test:
    img_num=data[1]
    img_data=data[0]

    img_data=img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    result=model.predict([img_data])[0]
    
    if result[0]==0:
      f.write(f'{img_num}, dog\n')
    else:
      f.write(f'{img_num}, cat\n')